**TODO**

https://www.kaggle.com/rtatman/analyzing-multilingual-data

https://github.com/cjhutto/vaderSentiment

In this notebook I'll try to add some more days to the last retrieved data from some data that I retrieved when building the code few days before the final version of [notebooks/01-collecting_and_saving_tweets.ipynb](http://localhost:8888/notebooks/twitter_analysis_online_grocery_NL/notebooks/01-collecting_and_saving_tweets.ipynb).

**TO DO**

- List goals notebook
- Talk about 4 tools to detect language
- show tradeoff (check my mutilingual tutorial notebook)
- build a function that apply all 3 (not the online one) and use majority code to choose language
- Apply to complete info about language
- combine info to make AH info goes from 30/03/2020 until 24/06/2020
- Save this in csv

# Goals of this notebook

# Load Packages

In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time

TodaysDate = time.strftime("%Y-%m-%d")

# Trying to add more days to Jumbo and AH

In [2]:
def search_file_in_folder(folder,str_file, type_file='csv'):
    """ Given a folder and a part of file's name outputs a list of files paths"""
    
    list_files_paths = []
    for file_path in glob.glob(folder+'*'+str_file+'*.'+type_file):
        try:
            list_files_paths.append(file_path)
        except:
            pass
    
    if len(list_files_paths):
        return list_files_paths
    else:
        return 'No files containing {}'.format(str_file)

In [3]:
def create_dataframe_info(result):
    """ Create a dataframe with tweet's data with path of the .csv file, min and max create_at date, 
    number of tweets, and number of columns """
    
    filepath_list = []
    min_created_list = []
    max_created_list = []
    n_tweet_list = []
    n_columns = []
    
    for file in result:
        df = pd.read_csv(file)
        df['created_at'] = pd.to_datetime(df['created_at'], infer_datetime_format=True)
        filepath_list.append(file)
        min_created_list.append(min(df['created_at']))
        max_created_list.append(max(df['created_at']))
        n_tweet_list.append(df.shape[0])
        n_columns.append(df.shape[1])
        
    dict_df = {'file_path':filepath_list,
              'min_created_list':min_created_list,
              'max_created_list':max_created_list,
              'n_tweet_list':n_tweet_list,
              'n_columns':n_columns}
    
    df_new = pd.DataFrame(dict_df)
                
    return df_new

In [4]:
folder = "../data/tweets/"
result = search_file_in_folder(folder, 'JumboSupermarkt')
result

['../data/tweets\\JumboSupermarkt_2020-06-16-21-20.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-16-21-21.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-16-21-23.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-16-21-35.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-16-21-56.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-21-00-31.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-21-00-33.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-21-00-34.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-21-13-17.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-21-13-18.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-22-14-59.csv',
 '../data/tweets\\JumboSupermarkt_2020-06-22-15-01.csv']

In [5]:
df_info_Jumbo = create_dataframe_info(result).sort_values(by=['min_created_list','n_tweet_list'])
df_info_Jumbo

,file_path,min_created_list,max_created_list,n_tweet_list,n_columns
1,../data/tweets\JumboSupermarkt_2020-06-16-21-2...,2020-03-03 17:46:39+00:00,2020-06-16 18:29:48+00:00,3235,14
2,../data/tweets\JumboSupermarkt_2020-06-16-21-2...,2020-03-03 17:46:39+00:00,2020-06-16 18:29:48+00:00,3235,14
3,../data/tweets\JumboSupermarkt_2020-06-16-21-3...,2020-03-03 17:46:39+00:00,2020-06-16 18:29:48+00:00,3235,14
4,../data/tweets\JumboSupermarkt_2020-06-16-21-5...,2020-03-04 14:45:25+00:00,2020-06-16 18:29:48+00:00,3217,14
5,../data/tweets\JumboSupermarkt_2020-06-21-00-3...,2020-03-06 19:12:05+00:00,2020-06-20 19:51:11+00:00,3221,19
7,../data/tweets\JumboSupermarkt_2020-06-21-00-3...,2020-03-06 19:12:05+00:00,2020-06-20 19:51:11+00:00,3221,19
8,../data/tweets\JumboSupermarkt_2020-06-21-13-1...,2020-03-06 19:12:05+00:00,2020-06-21 09:33:32+00:00,3230,19
6,../data/tweets\JumboSupermarkt_2020-06-21-00-3...,2020-03-07 20:15:43+00:00,2020-06-20 19:51:11+00:00,3203,19
9,../data/tweets\JumboSupermarkt_2020-06-21-13-1...,2020-03-07 20:15:43+00:00,2020-06-21 09:33:32+00:00,3212,19
10,../data/tweets\JumboSupermarkt_2020-06-22-14-5...,2020-03-07 20:15:43+00:00,2020-06-22 11:03:38+00:00,3232,19


In [6]:
df_info_Jumbo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 1 to 0
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   file_path         12 non-null     object             
 1   min_created_list  12 non-null     datetime64[ns, UTC]
 2   max_created_list  12 non-null     datetime64[ns, UTC]
 3   n_tweet_list      12 non-null     int64              
 4   n_columns         12 non-null     int64              
dtypes: datetime64[ns, UTC](2), int64(2), object(1)
memory usage: 576.0+ bytes


If I concatenate the most recent file with any of the 3 first files displayed in the dataframe I'm able to go back until 3rd March. One tradeoff is that the older data has less columns than the newer. Let's see what we can do about AH data.

In [7]:
folder = "../data/tweets/"
result = search_file_in_folder(folder, 'albertheijn')
result

['../data/tweets\\albertheijn_2020-06-16-21-58.csv',
 '../data/tweets\\albertheijn_2020-06-16-22-00.csv',
 '../data/tweets\\albertheijn_2020-06-16-22-18.csv',
 '../data/tweets\\albertheijn_2020-06-21-00-49.csv',
 '../data/tweets\\albertheijn_2020-06-21-00-53.csv',
 '../data/tweets\\albertheijn_2020-06-21-14-42.csv',
 '../data/tweets\\albertheijn_2020-06-21-14-43.csv',
 '../data/tweets\\albertheijn_2020-06-21-14-44.csv',
 '../data/tweets\\albertheijn_2020-06-21-14-45.csv',
 '../data/tweets\\albertheijn_2020-06-22-15-02.csv',
 '../data/tweets\\albertheijn_2020-06-22-15-03.csv']

In [8]:
df_info_AH = create_dataframe_info(result).sort_values(by=['min_created_list','n_tweet_list'])
df_info_AH

,file_path,min_created_list,max_created_list,n_tweet_list,n_columns
0,../data/tweets\albertheijn_2020-06-16-21-58.csv,2020-03-30 08:04:02+00:00,2020-06-16 16:36:50+00:00,3243,14
1,../data/tweets\albertheijn_2020-06-16-22-00.csv,2020-03-30 08:04:02+00:00,2020-06-16 16:36:50+00:00,3243,14
2,../data/tweets\albertheijn_2020-06-16-22-18.csv,2020-03-30 08:04:02+00:00,2020-06-16 16:36:50+00:00,3243,14
3,../data/tweets\albertheijn_2020-06-21-00-49.csv,2020-04-03 15:35:47+00:00,2020-06-20 18:18:50+00:00,3204,19
4,../data/tweets\albertheijn_2020-06-21-00-53.csv,2020-04-03 15:35:47+00:00,2020-06-20 18:18:50+00:00,3204,19
6,../data/tweets\albertheijn_2020-06-21-14-43.csv,2020-04-03 15:35:47+00:00,2020-06-21 12:27:32+00:00,3218,19
7,../data/tweets\albertheijn_2020-06-21-14-44.csv,2020-04-03 15:35:47+00:00,2020-06-21 12:27:32+00:00,3218,19
8,../data/tweets\albertheijn_2020-06-21-14-45.csv,2020-04-03 15:35:47+00:00,2020-06-21 12:45:16+00:00,3219,19
9,../data/tweets\albertheijn_2020-06-22-15-02.csv,2020-04-06 16:33:02+00:00,2020-06-22 11:43:16+00:00,3217,19
10,../data/tweets\albertheijn_2020-06-22-15-03.csv,2020-04-06 16:33:02+00:00,2020-06-22 11:43:16+00:00,3217,19


In [9]:
df_info_AH.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 5
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   file_path         11 non-null     object             
 1   min_created_list  11 non-null     datetime64[ns, UTC]
 2   max_created_list  11 non-null     datetime64[ns, UTC]
 3   n_tweet_list      11 non-null     int64              
 4   n_columns         11 non-null     int64              
dtypes: datetime64[ns, UTC](2), int64(2), object(1)
memory usage: 528.0+ bytes


Considering AH concatenating one of the 3 first files (see previous dataframe) with newer data we can go back to 30th March.

So let's check the columns we need to drop and concatenate the newer and older csv to increase our range from 30th March 2020 until 22nd June 2020.

In [10]:
# loading older data (back to 30th March)
df_AH_2020_06_16 = pd.read_csv(df_info_AH.loc[2,'file_path'])
# before checking for difference in the columns between old and new data I'll rename handle to screen_name since both are the same
df_AH_2020_06_16.rename(columns={'handle':'screen_name'},inplace=True)
df_AH_2020_06_16.head()

,mined_at,screen_name,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,source,text
0,2020-06-16 22:17:43.708324,albertheijn,1244535843135672326,1244535843135672326,2020-03-30 08:04:02+00:00,2020,3,30,0,8,4,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...","@derots Voorraad is er genoeg, het is voor ons..."
1,2020-06-16 22:17:43.708324,albertheijn,1244538454890987523,1244538454890987523,2020-03-30 08:14:24+00:00,2020,3,30,0,8,14,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@MoniquevDBurgh We doen er alles aan om zoveel...
2,2020-06-16 22:17:43.708324,albertheijn,1244540668225126401,1244540668225126401,2020-03-30 08:23:12+00:00,2020,3,30,0,8,23,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@AnnekeVisser15 Klopt! De Persoonlijke Bonus w...
3,2020-06-16 22:17:43.519106,albertheijn,1244541424588251141,1244541424588251141,2020-03-30 08:26:12+00:00,2020,3,30,0,8,26,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@waltervantiel We kopen groenten en fruit z...
4,2020-06-16 22:17:43.519106,albertheijn,1244542564344238083,1244542564344238083,2020-03-30 08:30:44+00:00,2020,3,30,0,8,30,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@bbstring Je kunt ons het beste een privéberic...


In [11]:
df_AH_2020_06_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3243 entries, 0 to 3242
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   mined_at       3243 non-null   object
 1   screen_name    3243 non-null   object
 2   tweet_id       3243 non-null   int64 
 3   tweet_id_str   3243 non-null   int64 
 4   created_at     3243 non-null   object
 5   year           3243 non-null   int64 
 6   month          3243 non-null   int64 
 7   day            3243 non-null   int64 
 8   day_of_week    3243 non-null   int64 
 9   hour           3243 non-null   int64 
 10  minute         3243 non-null   int64 
 11  retweet_count  3243 non-null   int64 
 12  source         3243 non-null   object
 13  text           3243 non-null   object
dtypes: int64(9), object(5)
memory usage: 354.8+ KB


In [12]:
# changing data type of 'created_at' to datetime

df_AH_2020_06_16['created_at'] = pd.to_datetime(df_AH_2020_06_16['created_at'], infer_datetime_format=True)

In [13]:
min(df_AH_2020_06_16['created_at']),max(df_AH_2020_06_16['created_at'])

(Timestamp('2020-03-30 08:04:02+0000', tz='UTC'),
 Timestamp('2020-06-16 16:36:50+0000', tz='UTC'))

Now we load the most recent data (until 22nd June).

In [14]:
df_AH_2020_06_22 = pd.read_csv(df_info_AH.loc[10,'file_path'])
df_AH_2020_06_22.head()

,mined_at,created_at,year,month,day,day_of_week,hour,minute,screen_name,tweet_id,tweet_id_str,retweet_count,favorite_count,source,language,user_favourites_count,followers_count,friends_count,text
0,2020-06-22 15:03:36.106324,2020-04-06 16:33:02+00:00,2020,4,6,0,16,33,albertheijn,1247200655002959875,1247200655002959875,0,NaN,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",nl,580,45532,6,@babbelaar4life Ik geef het door! Jij bedankt ...
1,2020-06-22 15:03:36.106324,2020-04-06 16:41:10+00:00,2020,4,6,0,16,41,albertheijn,1247202701689212931,1247202701689212931,0,NaN,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",fi,580,45532,6,@myrna_1995 Graag gedaan 😊 ^Stéphanie
2,2020-06-22 15:03:36.106324,2020-04-06 16:50:17+00:00,2020,4,6,0,16,50,albertheijn,1247204994442563584,1247204994442563584,0,NaN,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",nl,580,45532,6,@rliessum Hoi! We willen de beschikbare produc...
3,2020-06-22 15:03:36.106324,2020-04-06 16:59:09+00:00,2020,4,6,0,16,59,albertheijn,1247207225204105221,1247207225204105221,0,NaN,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",nl,580,45532,6,@peter_vinke Hi Peter! Bedankt voor jouw opmer...
4,2020-06-22 15:03:36.106324,2020-04-06 16:59:39+00:00,2020,4,6,0,16,59,albertheijn,1247207352492834819,1247207352492834819,0,NaN,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",nl,580,45532,6,@Lupardi Jij bent er als de kippen bij! Het ka...


In [15]:
df_AH_2020_06_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3217 entries, 0 to 3216
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   mined_at               3217 non-null   object 
 1   created_at             3217 non-null   object 
 2   year                   3217 non-null   int64  
 3   month                  3217 non-null   int64  
 4   day                    3217 non-null   int64  
 5   day_of_week            3217 non-null   int64  
 6   hour                   3217 non-null   int64  
 7   minute                 3217 non-null   int64  
 8   screen_name            3217 non-null   object 
 9   tweet_id               3217 non-null   int64  
 10  tweet_id_str           3217 non-null   int64  
 11  retweet_count          3217 non-null   int64  
 12  favorite_count         25 non-null     float64
 13  source                 3217 non-null   object 
 14  language               3217 non-null   object 
 15  user

In [16]:
df_AH_2020_06_22['created_at'] = pd.to_datetime(df_AH_2020_06_22['created_at'], infer_datetime_format=True)

In [17]:
df_AH_2020_06_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3217 entries, 0 to 3216
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   mined_at               3217 non-null   object             
 1   created_at             3217 non-null   datetime64[ns, UTC]
 2   year                   3217 non-null   int64              
 3   month                  3217 non-null   int64              
 4   day                    3217 non-null   int64              
 5   day_of_week            3217 non-null   int64              
 6   hour                   3217 non-null   int64              
 7   minute                 3217 non-null   int64              
 8   screen_name            3217 non-null   object             
 9   tweet_id               3217 non-null   int64              
 10  tweet_id_str           3217 non-null   int64              
 11  retweet_count          3217 non-null   int64            

## Which columns are present in both older and most recent data?

In [18]:
common_columns = list(set(df_AH_2020_06_16.columns).intersection(set(df_AH_2020_06_22.columns)))
common_columns.sort()
common_columns

['created_at',
 'day',
 'day_of_week',
 'hour',
 'mined_at',
 'minute',
 'month',
 'retweet_count',
 'screen_name',
 'source',
 'text',
 'tweet_id',
 'tweet_id_str',
 'year']

In [19]:
# this agrees with what we expected
len(common_columns)

14

## Which ones are missing in the older data?

The columns we will miss by concatenation the older and newer columns are:

In [20]:
list(set(df_AH_2020_06_22.columns).difference(set(df_AH_2020_06_16.columns)))

['user_favourites_count',
 'friends_count',
 'favorite_count',
 'language',
 'followers_count']

When concatenation I'll keep all columns since `friends_count`, `user_favourites_count`, `favorite_count`, `followers_count` can be used in EDA only considering dates available in the most recent data. 

For the sentiment analysis part `language` is a important piece of information. In addition, I want to maximize the amount of data available within COVID-19 period. Therefore, before concatenating I'll insert this information to the data retrieved on 16th June.

Because there is a overlap in period covered by the 2 dataframes I'll first keep only the days present in the data retrieve on 16th June that is not present in the most recent data retrieve at 22nd June. This will save us processing time.

In [21]:
# Keeping only days not in the most recent data
df_AH_2020_06_16 = df_AH_2020_06_16[df_AH_2020_06_16['created_at'] <= min(df_AH_2020_06_22['created_at'])]

In [22]:
df_AH_2020_06_16.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 204
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   mined_at       205 non-null    object             
 1   screen_name    205 non-null    object             
 2   tweet_id       205 non-null    int64              
 3   tweet_id_str   205 non-null    int64              
 4   created_at     205 non-null    datetime64[ns, UTC]
 5   year           205 non-null    int64              
 6   month          205 non-null    int64              
 7   day            205 non-null    int64              
 8   day_of_week    205 non-null    int64              
 9   hour           205 non-null    int64              
 10  minute         205 non-null    int64              
 11  retweet_count  205 non-null    int64              
 12  source         205 non-null    object             
 13  text           205 non-null    object             

Therefore, we have 205 new data points from 30th March until 6th April.

In [23]:
min(df_AH_2020_06_16['created_at']),max(df_AH_2020_06_16['created_at'])

(Timestamp('2020-03-30 08:04:02+0000', tz='UTC'),
 Timestamp('2020-04-06 16:33:02+0000', tz='UTC'))

In [24]:
min(df_AH_2020_06_22['created_at'])

Timestamp('2020-04-06 16:33:02+0000', tz='UTC')

## Insert `language` information to older data (retrieved on 16th June 2020)

We can consider one of the two solutions to insert information about the language in this older data.

1. [detect_language() from TextBlob()](https://textblob.readthedocs.io/en/dev/api_reference.html#textblob.blob.TextBlob.detect_language). It uses a Google API and requires Internet access.

I've tried but it didn't allow to have all 205 entries evaluated. I've got a `HTTPError: HTTP Error 429: Too Many Requests`

2. [langdetect Python library](https://pypi.org/project/langdetect/): Port of [Google’s language-detection library](https://code.google.com/archive/p/language-detection/) to Python.

Let's first consider an example and both ways of finding out language:

In [25]:
text = df_AH_2020_06_16.loc[0,'text']
text

'@derots Voorraad is er genoeg, het is voor ons echter een logistieke uitdaging om alles op het juiste moment op de juiste plek te hebben. Hier werken we dag en nacht en met man en macht aan om voor elkaar te krijgen. Sorry dat dit alsnog een leeg schap opleverd. ^Job'

In [26]:
# from textblob import TextBlob

# b = TextBlob(text)
# b.detect_language()

In [27]:
from langdetect import detect

detect(text)

'nl'

As we can see both `detect_language` from `TextBlob` and `langdetect` detected successfully the language of the dutch tweet. But if you have a good amount of text to work with better to use `langdected` that does not depend on the API.

In [30]:
df_AH_2020_06_16['language'] = df_AH_2020_06_22['text'].apply(lambda x: detect(x))

In [31]:
df_AH_2020_06_16.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 204
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   mined_at       205 non-null    object             
 1   screen_name    205 non-null    object             
 2   tweet_id       205 non-null    int64              
 3   tweet_id_str   205 non-null    int64              
 4   created_at     205 non-null    datetime64[ns, UTC]
 5   year           205 non-null    int64              
 6   month          205 non-null    int64              
 7   day            205 non-null    int64              
 8   day_of_week    205 non-null    int64              
 9   hour           205 non-null    int64              
 10  minute         205 non-null    int64              
 11  retweet_count  205 non-null    int64              
 12  source         205 non-null    object             
 13  text           205 non-null    object             

In [32]:
df_AH_2020_06_16.head()

,mined_at,screen_name,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,source,text,language
0,2020-06-16 22:17:43.708324,albertheijn,1244535843135672326,1244535843135672326,2020-03-30 08:04:02+00:00,2020,3,30,0,8,4,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...","@derots Voorraad is er genoeg, het is voor ons...",nl
1,2020-06-16 22:17:43.708324,albertheijn,1244538454890987523,1244538454890987523,2020-03-30 08:14:24+00:00,2020,3,30,0,8,14,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@MoniquevDBurgh We doen er alles aan om zoveel...,af
2,2020-06-16 22:17:43.708324,albertheijn,1244540668225126401,1244540668225126401,2020-03-30 08:23:12+00:00,2020,3,30,0,8,23,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@AnnekeVisser15 Klopt! De Persoonlijke Bonus w...,nl
3,2020-06-16 22:17:43.519106,albertheijn,1244541424588251141,1244541424588251141,2020-03-30 08:26:12+00:00,2020,3,30,0,8,26,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@waltervantiel We kopen groenten en fruit z...,nl
4,2020-06-16 22:17:43.519106,albertheijn,1244542564344238083,1244542564344238083,2020-03-30 08:30:44+00:00,2020,3,30,0,8,30,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@bbstring Je kunt ons het beste een privéberic...,nl


In [33]:
df_AH_2020_06_16['language'].value_counts()

nl    187
af      8
en      4
fr      1
tr      1
ro      1
no      1
so      1
de      1
Name: language, dtype: int64

In [34]:
df_AH_2020_06_16['text'][df_AH_2020_06_16['language']=='af'].values

array(['@MoniquevDBurgh We doen er alles aan om zoveel mogelijk bezorg- en afhaalmomenten vrij te krijgen. Het kan zijn dat een bezorgmoment al vol zit al vorens je deze kunt inzien. Klanten met een Bezorgbundel kunnen namelijk een week verder inplannen. ^Job',
       '@salvatore1976cs  Ik ga er vanuit dat dat op korte termijn zeker goedkomt 😉 ^Robin',
       '@moniquekauffman Het enige wat ik nu voor je kan doen is aanraden om de site goed in de gaten te houden en voor zover mogelijk gelijk een moment te reserveren als deze er weer zijn. ^Lies',
       '@jantine50 .@jantine We willen zo min mogelijk producten weggooien (want voedselverspilling), bij gebruik van zo min mogelijk verpakkingsmateriaal. Hier lees je meer over onze aanpak https://t.co/wAmLYVTBPm',
       '@KathyMamadottir Zoek een product daarom altijd op onze eigen website en niet via een zoekmachine. Een zoekmachine als Google slaat informatie op voor een bepaalde periode. Als wij een artikel verwijderen, duurt het even v

In [36]:
df_AH_2020_06_16['language'][df_AH_2020_06_16['language']=='af']='nl'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
df_AH_2020_06_16['language'].value_counts()

nl    195
en      4
fr      1
tr      1
ro      1
no      1
so      1
de      1
Name: language, dtype: int64

In [38]:
df_AH_2020_06_16['text'][df_AH_2020_06_16['language']=='en'].values

array(['@Annastasia_r Oei! De burger hoef je (gelukkig) niet mee te nemen. Laat je dit berichtje of de bon aan de servicebalie zien? ☺️ ^Yasmine',
       '@yvonnestaal Dat klinkt niet zo best! Graag hoor ik van jou in een privéberichtje om welke winkel het gaat en wat precies het probleem is. ^Daniël',
       '@LizHartevelt Graag gedaan 😊 ^Stéphanie',
       '@huhesas Hmm, smakelijk eten alvast 😊 ^Stéphanie'], dtype=object)

In [39]:
df_AH_2020_06_16['text'][df_AH_2020_06_16['language']=='fr'].values

array(['@demama Je krijgt dan de Bonus over de wél geleverde producten! Dit kun je controleren op de afleverbon. Mocht dit alsnog niet goed zijn gegaan, stuur ons dan een DM met bestelgegevens. Dan helpen we je verder :) ^Job'],
      dtype=object)

In [40]:
df_AH_2020_06_16['text'][df_AH_2020_06_16['language']=='no'].values

array(['@MirteDamen Het is momenteel inderdaad erg druk bij onze telefonische klantenservice. Misschien kunnen wij je wel helpen, kun je ons in een pb sturen waar het over gaat? ^Stéphanie'],
      dtype=object)

In [41]:
df_AH_2020_06_16['text'][df_AH_2020_06_16['language']=='id'].values

array([], dtype=object)

In [42]:
df_AH_2020_06_16['text'][df_AH_2020_06_16['language']=='tr'].values

array(['@sandervh1998  Oei, dit hoort inderdaad niet. Ik raad je aan even terug te gaan naar de betreffende AH en dit voor te leggen bij de servicebalie.  ^Robin'],
      dtype=object)

In [43]:
df_AH_2020_06_16['text'][df_AH_2020_06_16['language']=='pl'].values

array([], dtype=object)

In [44]:
from langdetect import detect_langs
detect_langs('@NancyGroningen Oh nee! Excuses hiervoor! Zou je mij het bestelnummer of het e-mailadres in een privébericht willen sturen? Dan kijk ik mee ^Yasmine')

[nl:0.9999959158555007]

It seems that all `text` are Dutch and for some reason the detection failed in some cases. So we will change everything to `nl`. 

In [45]:
df_AH_2020_06_16['language'] = 'nl'

## Concatenating older and most recent data

Now we are ready to concatenate all AH data.

In [46]:
# concatenate dataframes
df_AH_concat = pd.concat([df_AH_2020_06_16,df_AH_2020_06_22])

In [47]:
duplicateRowsDF = df_AH_concat[df_AH_concat.duplicated(['text'])]
duplicateRowsDF

,mined_at,screen_name,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,source,text,language,favorite_count,user_favourites_count,followers_count,friends_count
0,2020-06-22 15:03:36.106324,albertheijn,1247200655002959875,1247200655002959875,2020-04-06 16:33:02+00:00,2020,4,6,0,16,33,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@babbelaar4life Ik geef het door! Jij bedankt ...,nl,NaN,580.0,45532.0,6.0


As expected tweet from '2020-04-06 16:33:02+0000' is duplicated. It will be removed by using `drop_duplicates`.

In [48]:
df_AH_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3422 entries, 0 to 3216
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   mined_at               3422 non-null   object             
 1   screen_name            3422 non-null   object             
 2   tweet_id               3422 non-null   int64              
 3   tweet_id_str           3422 non-null   int64              
 4   created_at             3422 non-null   datetime64[ns, UTC]
 5   year                   3422 non-null   int64              
 6   month                  3422 non-null   int64              
 7   day                    3422 non-null   int64              
 8   day_of_week            3422 non-null   int64              
 9   hour                   3422 non-null   int64              
 10  minute                 3422 non-null   int64              
 11  retweet_count          3422 non-null   int64            

In [49]:
# eliminate duplicates based on create_at and text, keep will be setted to 'last' since we know that in the older 
# data language will be nan and it is better to keep data that is not nan

df_AH_concat.drop_duplicates(subset=['text'], inplace = True, keep = 'last')

In [50]:
# sorting by 'created_at'
df_AH_concat.sort_values(by='created_at',inplace = True)

# reset index
df_AH_concat.reset_index(drop = True, inplace = True)

# save in csv

df_AH_concat.to_csv("../data/processed/AH_concat_16_and_22_June_"+TodaysDate+".csv", index = False)

In [51]:
df_AH_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421 entries, 0 to 3420
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   mined_at               3421 non-null   object             
 1   screen_name            3421 non-null   object             
 2   tweet_id               3421 non-null   int64              
 3   tweet_id_str           3421 non-null   int64              
 4   created_at             3421 non-null   datetime64[ns, UTC]
 5   year                   3421 non-null   int64              
 6   month                  3421 non-null   int64              
 7   day                    3421 non-null   int64              
 8   day_of_week            3421 non-null   int64              
 9   hour                   3421 non-null   int64              
 10  minute                 3421 non-null   int64              
 11  retweet_count          3421 non-null   int64            

In [52]:
df_AH_concat.head()

,mined_at,screen_name,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,source,text,language,favorite_count,user_favourites_count,followers_count,friends_count
0,2020-06-16 22:17:43.708324,albertheijn,1244535843135672326,1244535843135672326,2020-03-30 08:04:02+00:00,2020,3,30,0,8,4,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...","@derots Voorraad is er genoeg, het is voor ons...",nl,NaN,NaN,NaN,NaN
1,2020-06-16 22:17:43.708324,albertheijn,1244538454890987523,1244538454890987523,2020-03-30 08:14:24+00:00,2020,3,30,0,8,14,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@MoniquevDBurgh We doen er alles aan om zoveel...,nl,NaN,NaN,NaN,NaN
2,2020-06-16 22:17:43.708324,albertheijn,1244540668225126401,1244540668225126401,2020-03-30 08:23:12+00:00,2020,3,30,0,8,23,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@AnnekeVisser15 Klopt! De Persoonlijke Bonus w...,nl,NaN,NaN,NaN,NaN
3,2020-06-16 22:17:43.519106,albertheijn,1244541424588251141,1244541424588251141,2020-03-30 08:26:12+00:00,2020,3,30,0,8,26,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@waltervantiel We kopen groenten en fruit z...,nl,NaN,NaN,NaN,NaN
4,2020-06-16 22:17:43.519106,albertheijn,1244542564344238083,1244542564344238083,2020-03-30 08:30:44+00:00,2020,3,30,0,8,30,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@bbstring Je kunt ons het beste een privéberic...,nl,NaN,NaN,NaN,NaN


In [53]:
df_AH_concat.tail()

,mined_at,screen_name,tweet_id,tweet_id_str,created_at,year,month,day,day_of_week,hour,minute,retweet_count,source,text,language,favorite_count,user_favourites_count,followers_count,friends_count
3416,2020-06-22 15:03:31.445447,albertheijn,1275016605689315341,1275016605689315341,2020-06-22 10:43:42+00:00,2020,6,22,0,10,43,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...","@asjemenou Ai, ik kan niet precies zien wat he...",nl,NaN,580.0,45532.0,6.0
3417,2020-06-22 15:03:31.445447,albertheijn,1275017248567111684,1275017248567111684,2020-06-22 10:46:15+00:00,2020,6,22,0,10,46,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...","@malmostoso Hi Fabio, oei dat is inderdaad gee...",nl,NaN,580.0,45532.0,6.0
3418,2020-06-22 15:03:31.445447,albertheijn,1275024666260840448,1275024666260840448,2020-06-22 11:15:43+00:00,2020,6,22,0,11,15,0,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@malmostoso Graag gedaan natuurlijk! ^Dylon ht...,nl,NaN,580.0,45532.0,6.0
3419,2020-06-22 15:03:31.445447,albertheijn,1275029563542851585,1275029563542851585,2020-06-22 11:35:11+00:00,2020,6,22,0,11,35,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@Claire94175123 Hi! Helaas kunnen tickets nie...,nl,NaN,580.0,45532.0,6.0
3420,2020-06-22 15:03:31.445447,albertheijn,1275031595708276736,1275031595708276736,2020-06-22 11:43:16+00:00,2020,6,22,0,11,43,0,"<a href=""https://www.tracebuzz.com"" rel=""nofol...",@RhidoInfo Hoi. Vanaf maandag 22 juni krijgt d...,nl,NaN,580.0,45532.0,6.0


In [54]:
df_test = pd.read_csv("../data/processed/AH_concat_16_and_22_June_"+TodaysDate+".csv")

In [55]:
df_test.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421 entries, 0 to 3420
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   mined_at               3421 non-null   object 
 1   screen_name            3421 non-null   object 
 2   tweet_id               3421 non-null   int64  
 3   tweet_id_str           3421 non-null   int64  
 4   created_at             3421 non-null   object 
 5   year                   3421 non-null   int64  
 6   month                  3421 non-null   int64  
 7   day                    3421 non-null   int64  
 8   day_of_week            3421 non-null   int64  
 9   hour                   3421 non-null   int64  
 10  minute                 3421 non-null   int64  
 11  retweet_count          3421 non-null   int64  
 12  source                 3421 non-null   object 
 13  text                   3421 non-null   object 
 14  language               3421 non-null   object 
 15  favo

# Conclusions

